# Introduction to NLP fundamental using tensorflow
Deriving information from text or speech

or sequence 2 sequence generation

In [1]:
# !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
from helper_functions import unzip_data,create_tensorboard_callback, plot_loss_curves, compare_historys

## Get Data
twitter disaster

In [2]:
# !wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-03-22 20:06:00--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.193.128, 209.85.202.128, 172.253.116.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.193.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  1.10MB/s    in 0.5s    

2023-03-22 20:06:01 (1.10 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [3]:
# unzip_data("nlp_getting_started.zip")

# Visualize, Visualize, Visualize

can use panda (limited by ram)
using tf load_text
using python loader

In [6]:
import pandas as pd

train_df = pd.read_csv("data/nlp_getting_started/train.csv")
test_df = pd.read_csv("data/nlp_getting_started/test.csv")

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [19]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

## split into validation

In [23]:
from sklearn.model_selection import train_test_split

train_text, val_text, train_label, val_label = train_test_split(train_df_shuffled["text"].to_numpy(),
                                train_df_shuffled["target"].to_numpy(), test_size=.1, random_state=42)

In [24]:
len(train_text), len(train_label), len(val_text), len(val_label)

(6851, 6851, 762, 762)

In [26]:
train_text[:10], train_label[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Vectorizing words


In [30]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [33]:
round(sum([len(i.split()) for i in train_text])/len(train_text)) # avg length of text

15

In [34]:
max_vocab_length =10000
max_length = 15

In [40]:
textvectorization = TextVectorization(max_tokens=max_vocab_length # length of volcabulary(rest to <OOV>)
                                      , standardize='lower_and_strip_punctuation',
    split='whitespace',ngrams=None,# ngram size
                                      output_mode='int', # to numbers
                                      output_sequence_length=max_length,# pad to max length
                                      pad_to_max_tokens=True  
                                     )

In [41]:
textvectorization.adapt(train_text)

2023-03-23 07:07:23.722182: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [42]:
sample = "There's a flood in my street"
textvectorization([sample])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [43]:
import random

In [48]:
random_choice = random.choice(train_text)
print(random_choice)
print(textvectorization([random_choice]))

#NP Metallica - Damage Inc
tf.Tensor(
[[2836    1  233 5376    0    0    0    0    0    0    0    0    0    0
     0]], shape=(1, 15), dtype=int64)


In [52]:
our_vocabulary = textvectorization.get_vocabulary()
len(our_vocabulary), our_vocabulary[:5], our_vocabulary[-5:]

(10000,
 ['', '[UNK]', 'the', 'a', 'in'],
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'])

# Creating embedding layer


In [56]:
embedding = tf.keras.layers.Embedding(
    input_dim=max_vocab_length,
    output_dim=128, # set /8 when using gpu
    embeddings_initializer='uniform',
    embeddings_regularizer=None,
    activity_regularizer=None,
    embeddings_constraint=None,
    mask_zero=False,
    input_length=max_length
)

embedding

In [66]:
random_choice = random.choice(train_text)
print(random_choice)
vector = textvectorization([random_choice])
print(vector)
sample_embed = embedding(vector)

Sharing to help our cousin's family http://t.co/LSJowGYvQh
tf.Tensor(
[[4647    5  148  103 5902  302    1    0    0    0    0    0    0    0
     0]], shape=(1, 15), dtype=int64)


In [69]:
sample_embed[0][0], sample_embed[0][0].shape, random_choice

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.03114157,  0.01967374, -0.04736068, -0.01796595, -0.04870046,
        -0.0001228 , -0.03656517,  0.02118117, -0.02806422,  0.02665761,
        -0.02381406, -0.02557567, -0.048534  ,  0.02123635,  0.04308787,
         0.015731  ,  0.02820578,  0.03733463,  0.01771598,  0.04169016,
         0.04608223,  0.00220302, -0.0491954 ,  0.02065032, -0.04925536,
        -0.02303215, -0.04448877,  0.01592794, -0.04267229, -0.00565269,
        -0.03344625,  0.03074105, -0.04407436,  0.00965048,  0.00337793,
        -0.02072367,  0.00022598, -0.00970417,  0.00411699,  0.01119586,
        -0.04921091,  0.02989355,  0.02834091,  0.04475928, -0.0474855 ,
        -0.04036152, -0.00068638, -0.03608254, -0.02498015, -0.03486028,
         0.02828873,  0.03030254,  0.04037792, -0.04536163,  0.01363734,
        -0.00410252, -0.0279858 ,  0.0090971 ,  0.02767198,  0.02126597,
        -0.04666856, -0.02895921, -0.01100662, -0.02405472,  0.0262517 ,
  

# Experiment list

- naive bayes with tf-idf
- dense model
- LSTM(RNN)
- GRU (RNN) (gated recurrent unit)
- Bidrectional-LSTM (RNN)
- 1d CNN
- TF pretrained model feature extractor
- TF pretrained model feature extractor (10% data)

In [75]:

model_0 = tf.keras.models.Sequential()
model_0.add(tf.keras.layers.Input(shape=(1,), dtype=tf.string, name="input_layer"))

model_0.add(textvectorization)

model_0.add(embedding)



In [76]:
random_choice = random.choice(train_text)
model_0.predict([random_choice])

1/1 [==============================] - 0s 78ms/step


2023-03-23 07:37:48.208123: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


array([[[ 0.03648401, -0.01343995,  0.02726519, ..., -0.01601354,
          0.0075595 , -0.03912764],
        [-0.00736792, -0.04412271,  0.03118983, ..., -0.01487361,
         -0.03070074,  0.04722324],
        [-0.03944248,  0.03668461, -0.02891522, ...,  0.01199384,
          0.03239999, -0.03269011],
        ...,
        [ 0.02208832,  0.04889195,  0.0108219 , ...,  0.00643492,
          0.00560063, -0.02738208],
        [ 0.03689495, -0.04735472,  0.01407543, ..., -0.03450272,
          0.04619033, -0.02157602],
        [-0.02808598, -0.01386461,  0.00114881, ...,  0.02699247,
          0.04818046, -0.01146827]]], dtype=float32)